In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc5'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

Using TensorFlow backend.


(2686650, 2) (1878528, 2) (1957461, 2) (2416238, 2)
reset_index 완료


,bin,label
0,55,1
1,48,0
2,89,0
3,e5,0
4,48,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2686650, 257) (1878528, 257) (1957461, 257) (2416238, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2682208
1       4442
Name: label, dtype: int64 
 0    1875181
1       3347
Name: label, dtype: int64
0    1954180
1       3281
Name: label, dtype: int64 
 0    2413218
1       3020
Name: label, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    563504
48    337916
8b    176237
ff    170570
89    115546
       ...  
93       238
b1       222
a6       206
9e       181
a9       180
Name: bin, Length: 256, dtype: int64

In [6]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
# left_idx, right_idx = -10, 11 # 10개씩
left_idx, right_idx = 0, 6 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 26651


In [7]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 19965


In [8]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 19654


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 18095


In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(26651, 256) (26651,)
(19965, 256) (19965,)
(19654, 256) (19654,)
(18095, 256) (18095,)


In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(26651, 1, 256) (26651, 1, 1)
(19965, 1, 256) (19965, 1, 1)
(19654, 1, 256) (19654, 1, 1)
(18095, 1, 256) (18095, 1, 1)


In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(84365, 1, 256) (84365, 1, 1)


In [13]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []


xInput = Input(batch_shape=(None, 1, 256)) # 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):

    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
75928/75928 [==============================] - 12s 158us/step - loss: 0.3442 - accuracy: 0.8399
Epoch 2/3
75928/75928 [==============================] - 9s 123us/step - loss: 0.3155 - accuracy: 0.8427
Epoch 3/3
75928/75928 [==============================] - 10s 125us/step - loss: 0.3151 - accuracy: 0.8428


accuracy_score 0.9986962190352021
recall_score 1.0
precision_score 0.9922371206774877
f1_score 0.9961034360609281
======Training stage======
Epoch 1/3
75928/75928 [==============================] - 9s 125us/step - loss: 0.3147 - accuracy: 0.8428
Epoch 2/3
75928/75928 [==============================] - 9s 121us/step - loss: 0.3146 - accuracy: 0.8428
Epoch 3/3
75928/75928 [==============================] - 6s 74us/step - loss: 0.3148 - accuracy: 0.8430


accuracy_score 0.9985776934929478
recall_score 1.0
precision_score 0.9915373765867419
f1_score 0.9957507082152974
======Training stage======
Epoch 1/3
75928/75928 [==============================] - 6s 73us/step - loss: 0.3147 - accuracy: 0.8430
Epoch 2/3
75928/75928 [==============================] - 5s 72us/step - loss: 0.3146 - accuracy: 0.8430
Epoch 3/3
75928/75928 [==============================] - 5s 71us/step - loss: 0.3145 - accuracy: 0.8430


accuracy_score 0.9985776934929478
recall_score 1.0
precision_score 0.9915373765867419
f1_score 0.9957507082152974
======Training stage======
Epoch 1/3
75928/75928 [==============================] - 6s 74us/step - loss: 0.2959 - accuracy: 0.8636
Epoch 2/3
75928/75928 [==============================] - 5s 71us/step - loss: 0.2956 - accuracy: 0.8635
Epoch 3/3
75928/75928 [==============================] - 5s 68us/step - loss: 0.2959 - accuracy: 0.8634


accuracy_score 0.8122555410691004
recall_score 0.2680851063829787
precision_score 0.4064516129032258
f1_score 0.3230769230769231
======Training stage======
Epoch 1/3
75928/75928 [==============================] - 5s 69us/step - loss: 0.2931 - accuracy: 0.8669
Epoch 2/3
75928/75928 [==============================] - 6s 80us/step - loss: 0.2933 - accuracy: 0.8668
Epoch 3/3
75928/75928 [==============================] - 6s 74us/step - loss: 0.2933 - accuracy: 0.8669


...

accuracy_score 0.7924371740161213
recall_score 0.12393767705382436
precision_score 0.2539912917271408
f1_score 0.1665873393622085
======Training stage======
Epoch 1/3
75929/75929 [==============================] - 8s 104us/step - loss: 0.2961 - accuracy: 0.8638
Epoch 2/3
75929/75929 [==============================] - 6s 83us/step - loss: 0.2960 - accuracy: 0.8638
Epoch 3/3
75929/75929 [==============================] - 6s 78us/step - loss: 0.2960 - accuracy: 0.8636


accuracy_score 0.811284969179706
recall_score 0.12845990063875087
precision_score 0.3321100917431193
f1_score 0.18526100307062435
======Training stage======
Epoch 1/3
75929/75929 [==============================] - 5s 71us/step - loss: 0.2965 - accuracy: 0.8650
Epoch 2/3
75929/75929 [==============================] - 5s 66us/step - loss: 0.2965 - accuracy: 0.8651
Epoch 3/3
75929/75929 [==============================] - 5s 65us/step - loss: 0.2965 - accuracy: 0.8649


accuracy_score 0.7999051683262209
recall_score 0.12295664534470505
precision_score 0.2759170653907496
f1_score 0.17010816125860373
======Training stage======
Epoch 1/3
75929/75929 [==============================] - 5s 66us/step - loss: 0.2960 - accuracy: 0.8649
Epoch 2/3
75929/75929 [==============================] - 5s 71us/step - loss: 0.2959 - accuracy: 0.8650
Epoch 3/3
75929/75929 [==============================] - 6s 83us/step - loss: 0.2958 - accuracy: 0.8648


accuracy_score 0.8003793266951161
recall_score 0.11647727272727272
precision_score 0.271523178807947
f1_score 0.16302186878727634
======Training stage======
Epoch 1/3
75929/75929 [==============================] - 6s 76us/step - loss: 0.2959 - accuracy: 0.8664
Epoch 2/3
75929/75929 [==============================] - 6s 79us/step - loss: 0.2958 - accuracy: 0.8665
Epoch 3/3
75929/75929 [==============================] - 6s 74us/step - loss: 0.2960 - accuracy: 0.8665


accuracy_score 0.7872214319582741
recall_score 0.11158493248045487
precision_score 0.22364672364672364
f1_score 0.14888572783309623

K-fold cross validation Accuracy: [0.9986962190352021, 0.9985776934929478, 0.9985776934929478, 0.8122555410691004, 0.7826241555055115, 0.7924371740161213, 0.811284969179706, 0.7999051683262209, 0.8003793266951161, 0.7872214319582741]

K-fold cross validation Recall: [1.0, 1.0, 1.0, 0.2680851063829787, 0.10247349823321555, 0.12393767705382436, 0.12845990063875087, 0.12295664534470505, 0.11647727272727272, 0.11158493248045487]

K-fold cross validation Precision: [0.9922371206774877, 0.9915373765867419, 0.9915373765867419, 0.4064516129032258, 0.20451339915373765, 0.2539912917271408, 0.3321100917431193, 0.2759170653907496, 0.271523178807947, 0.22364672364672364]

K-fold cross validation F1-Score: [0.9961034360609281, 0.9957507082152974, 0.9957507082152974, 0.3230769230769231, 0.13653483992467047, 0.1665873393622085, 0.18526100307062435, 0.17010816125860373, 0

In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.883337183618257
10-Fold Cross_validation. Recall : 0.071872541427323
10-Fold Cross_validation. Precision : 0.1002346695019233
10-Fold Cross_validation. F1-Score : 0.08347866589161287
